In [1]:
### Create Feature: Lag 6 years ago

In [ ]:
# Specify the number of years to lag
lag_years = 6

# Calculate the lagged date
df_train['lag_rain_1h_6_years_date'] = df_train['datetime_iso'] - pd.DateOffset(years=lag_years)
df_test['lag_rain_1h_6_years_date'] = df_test['datetime_iso'] - pd.DateOffset(years=lag_years)

df_train['lag_rain_1h_6_years_date'] = pd.to_datetime(df_train['lag_rain_1h_6_years_date'])
df_test['lag_rain_1h_6_years_date'] = pd.to_datetime(df_test['lag_rain_1h_6_years_date'])

# Merge the DataFrame with itself to get lag_rain_1h_6_years_value
result_df =df_train.merge(df_train[['datetime_iso', 'rain_1h']], left_on='lag_rain_1h_6_years_date', right_on='datetime_iso', suffixes=('', '_lag'))

# Rename the 'rain_1h_lag' column to 'lag_rain_1h_6_years_value'
result_df.rename(columns={'rain_1h_lag': 'lag_rain_1h_6_years_value'}, inplace=True)

# Drop the duplicated datetime_iso column
result_df.drop(columns='datetime_iso_lag', inplace=True)

In [ ]:
result_df.head()
result_df.drop(['lag_rain_1h_6_years_date'])